# Read MAPS Database

This notebook represents my initial Python development to read piano audio samples from the "MAPS Database" and prepare them to be used as training data for a pitch-detection neural net.

## The MAPS Database

The MAPS ("MIDI Aligned Piano Sounds") Database is a collection of piano sounds, ranging from individual notes and chords to complete pieces of music.

Each sound file is accompanied by the MIDI data that was used to produce it. This data acts as the "ground truth" for the sample.

MAPS was produced by Valentin Emiya (Telecom ParisTech, 2008), and is available at [Telecom ParisTech's website](http://www.tsi.telecom-paristech.fr/aao/en/2010/07/08/maps-database-a-piano-database-for-multipitch-estimation-and-automatic-transcription-of-music/).

## Layout of the Database Files

The database consists of a collection of directories and files.

The top level consists of a number of instruments, each having its own directory.

Within each instrument's directory, a well-defined directory structure exists. For example, a set of *isol*ated notes, played *no*rmally exist within the "ISOL/NO" directory for each instrument. This directory structure is documented in the "MAPS_doc.pdf" file included within the database.

## Initial Scope of Work

At first, I will only be dealing with individual notes. Multi-note training will be handled later.

---

## Setup

The `maps_path` variable needs to be set to the root directory of the MAPS database. This is the directory that contains the instrument directories.

In [1]:
import os
import scipy.io
from scipy.io import wavfile

maps_path = '/datasets/audio/maps'
isolated_notes_subpath = 'ISOL/NO'

## Instruments

`list_instruments` is a generator returning the name and directory of each instrument found in the database.

In [2]:
def list_instruments(root_directory):
    files_and_dirs = ( (file_or_dir, os.path.join(root_directory, file_or_dir)) for file_or_dir in os.listdir(root_directory))
    return ( {'instrument':name, 'directory': directory} for name, directory in files_and_dirs if os.path.isdir(directory))

[instrument for instrument in list_instruments(maps_path)]

[{'instrument': 'ENSTDkAm', 'directory': '/datasets/audio/maps\\ENSTDkAm'},
 {'instrument': 'StbgTGd2', 'directory': '/datasets/audio/maps\\StbgTGd2'}]

## Sample files

Sample audio data is stored in `.wav` files.

Each audio file is accompanied by a `.txt` file and a `.mid` file.

The `.txt` file contains a list (in CSV format) of the notes present in the file.

The `.mid` file is the MIDI file from which the audio was generated.

At the moment, I don't use the MIDI file, so a "sample" is defined as a pair of a `.wav` file and a `.txt` file with the same base name.

Given a directory (such as `<maps_root>/<instrument>/ISOL/NO`), `list_samples` generates a list of samples in that directory.

In [3]:
def list_samples(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            root, _ = os.path.splitext(filename)
            txt_filename = os.path.join(directory, root+'.txt')
            if os.path.isfile(txt_filename):
                yield {'sample':root, 'wav_file':os.path.join(directory, root+'.wav'), 'txt_file':txt_filename}
                
next(samples for samples in list_samples(os.path.join(maps_path, 'StbgTGd2', isolated_notes_subpath)))

{'sample': 'MAPS_ISOL_NO_F_S0_M100_StbgTGd2',
 'wav_file': '/datasets/audio/maps\\StbgTGd2\\ISOL/NO\\MAPS_ISOL_NO_F_S0_M100_StbgTGd2.wav',
 'txt_file': '/datasets/audio/maps\\StbgTGd2\\ISOL/NO\\MAPS_ISOL_NO_F_S0_M100_StbgTGd2.txt'}

## Read sample

Given an item from `list_samples`, such as:

    {'sample': 'MAPS_ISOL_NO_F_S0_M100_StbgTGd2',
     'wav_file': '<maps_root>/<instrument>ISOL/NO/MAPS_ISOL_NO_F_S0_M100_StbgTGd2.wav',
     'txt_file': '<maps_root>/<instrument>ISOL/NO/MAPS_ISOL_NO_F_S0_M100_StbgTGd2.txt'}

`read_sample` will read the audio data (using `read_sample_audio`) and the list of notes (using `read_sample_notes`), and will return an item such as :

In [4]:
def read_sample_audio(wav_filename):
    return scipy.io.wavfile.read(wav_filename)

import csv

def read_sample_notes(txt_filename):
    with open(txt_filename) as file:
        reader = csv.DictReader(file, dialect='excel-tab')
        return [{'onset':note['OnsetTime'], 'offset':note['OffsetTime'], 'midi_pitch':note['MidiPitch']} for note in reader]

def read_sample(sample):
    sample_rate, audio = read_sample_audio(sample['wav_file'])
    notes = read_sample_notes(sample['txt_file'])
    return {'sample': sample['sample'],
            'sample_rate': sample_rate,
            'audio': audio,
            'notes': notes
           }

read_sample({'sample': 'MAPS_ISOL_NO_F_S0_M100_StbgTGd2',
     'wav_file': os.path.join(maps_path, 'StbgTGd2', isolated_notes_subpath, 'MAPS_ISOL_NO_F_S0_M100_StbgTGd2.wav'),
     'txt_file': os.path.join(maps_path, 'StbgTGd2', isolated_notes_subpath, 'MAPS_ISOL_NO_F_S0_M100_StbgTGd2.txt')})

{'sample': 'MAPS_ISOL_NO_F_S0_M100_StbgTGd2',
 'sample_rate': 44100,
 'audio': array([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]], dtype=int16),
 'notes': [{'onset': '0.500004', 'offset': '2.50001', 'midi_pitch': '100'}]}

## Putting It All Together

`read_samples` is a generator that, given the MAPS root directory, will produce entries with the following form:

    instrument
    sample
    sample_rate
    audio
    notes

In [5]:
def read_samples(root_directory):
    for instrument in list_instruments(root_directory):
        for sample in list_samples(os.path.join(instrument['directory'], isolated_notes_subpath)):
            read = read_sample(sample)
            yield {'instrument': instrument['instrument'],
                   'sample': sample['sample'],
                   'sample_rate': read['sample_rate'],
                   'audio': read['audio'],
                   'notes': read['notes']
                  }
        
next(read_samples(maps_path))

{'instrument': 'ENSTDkAm',
 'sample': 'MAPS_ISOL_NO_F_S0_M100_ENSTDkAm',
 'sample_rate': 44100,
 'audio': array([[ 1,  9],
        [ 0,  7],
        [-1,  7],
        ...,
        [19, 19],
        [18, 17],
        [18, 18]], dtype=int16),
 'notes': [{'onset': '0.51599', 'offset': '2.521', 'midi_pitch': '100'}]}